In [2]:
import os
from PIL import Image
import pytesseract
import re
from pdf2image import convert_from_path

import logging
import sys

import pandas as pd

In [201]:
arvd_strs = "|".join(['Arvd~', 'Arvd-', "Arv@-"])
clrd_strs = "|".join(['Clrd-', 'Cird-', 'Clird-', 'Clrd+'])
taker_strs = "|".join(["‘Call Taker", "Call Taker", "Cail Taker", "Cali Taker"])
loc_strs = "|".join(["Location/Address", "Locatiion/Address", "Location"])

def find_between(entry_text, left_text, right_text):
    left_end = re.search(left_text, entry_text)
    right_start = re.search(right_text, entry_text)
    
    if left_end and right_start:
        return entry_text[left_end.end():right_start.start()].strip()
    else:
        return None

def find_next_word(entry_text, left_text):
    left_end = re.search(left_text, entry_text)
    
    if left_end:
        return re.search('([^\s]+)', entry_text[left_end.end():]).group(0)
    else:
        return None
    

In [217]:
string_tol = 60
        
def parse_entry(entry_text):
    entry_words = [w for w in entry_text.split(' ') if len(w) > 0]
    
    
    if len(entry_words) <2:
        return [None]*6
    else:    
        # call number is always the first word of the entry
        call_number = entry_words[0]

        # the next 'word' is always the time
        call_time = entry_words[1][:min(4, len(entry_words[1]))]
    
        # call reason
        call_reason =  find_between(entry_text, call_time, taker_strs)
        if call_reason is None:
            if re.search(call_time, entry_text) and re.search(taker_strs, entry_text) is None:
                print(re.search(call_time, entry_text), re.search(taker_strs, entry_text))
            #call_taker = entry_text[re.search(call_time, entry_text).end():]

        # call taker is always the string between taker and location
        call_taker =  find_between(entry_text, taker_strs, loc_strs)
        if call_taker is None and re.search(taker_strs, entry_text):
            # see if its actually near the end of the string so location doesnt appear
            if len(entry_text) - re.search(taker_strs, entry_text).end() < string_tol:
                call_taker = entry_text[re.search(taker_strs, entry_text).end():]
            else:
                print("End of string?", len(entry_text) - re.search(taker_strs, entry_text).end(), entry_text)
            

        arvd_time = find_next_word(entry_text, arvd_strs)
        clrd_time = find_next_word(entry_text, clrd_strs)
    
    return [call_number, call_time, call_reason, call_taker, arvd_time, clrd_time]


def replace_none_with_value(left_list, right_list):
    return [l if not l is None else r for l, r in zip(left_list, right_list)]


In [221]:
year_str = '19-'
current_date = '01/01/2019'

Arvd_list = []
Clrd_list = []


i_entry = 0
parsed_pages = []

for ipage in range(1,20):
    with open('../../data/Logs2019/page_{}.txt'.format(ipage),'r') as infile:
        page_text = infile.read()
    
    #print(ipage)
    #if ipage==2:
    #    print(page_text)
    
    
    page_entries = [log_idx.start() for log_idx in re.finditer(year_str, page_text)] + [-1]

    # check for updated date - we assume only once ever on the page
    date_end = [date_idx.end() for date_idx in re.finditer('For Date: ', page_text)]
    if len(date_end) > 1:
        print("Oh no, multiple days on the same page... this wont work.")
        adgasdgs
    
    # we found only one date so update
    elif len(date_end) == 1:
        current_date = re.search('([^\s]+)', page_text[date_end[0]:])
        if current_date:
            current_date = current_date.group(0).replace('-', "").strip()


    # now we worry about entries that start from the previous page
    initial_text = page_text[0:page_entries[0]]
    if len(initial_text) > 0 and len(parsed_pages) > 0:
        #print(initial_text)
        str_entry = parse_entry(initial_text)
        
        parsed_pages[-1][2:] = replace_none_with_value(parsed_pages[-1][2:], str_entry)
        
    
    for i_start in range(len(page_entries)-1):

        entry_text= page_text[page_entries[i_start]:page_entries[i_start + 1]]
        
        str_entry = parse_entry(entry_text)
        
        if str_entry.count(None) >= 5:
            # append text to previous narrative
            pass
        else:
            parsed_pages.append([current_date, ipage] + str_entry)
            i_entry += 1
            
        if str_entry[0] == '19-95':
            print(entry_text)
            print(str_entry)
        
        
        

<re.Match object; span=(12, 16), match='Remo'> None
<re.Match object; span=(6, 8), match='34'> None
<re.Match object; span=(7, 11), match='1738'> None
<re.Match object; span=(10, 13), match='ABI'> None
<re.Match object; span=(18, 22), match='COLD'> None
<re.Match object; span=(7, 9), match='36'> None
<re.Match object; span=(7, 9), match='36'> None
<re.Match object; span=(7, 11), match='1030'> None
<re.Match object; span=(7, 11), match='1719'> None
<re.Match object; span=(7, 11), match='1936'> None
<re.Match object; span=(6, 7), match='W'> None
End of string? 249 19-94  0807  Other ~ ASSIST OTHER AGENCY - REPO  SERVICES RENDERED  Cali Taker  PATROL DAVID JENNINGS, D  Vehicle  WHI 2013 HYUN SE ELANTRA Reg: PC MA 2CS815 VIN: 5NPDH4AE5DH351247  Owner  LAFLECHE, JANELL MARIE @ 15 ARCH ST - PITTSFIELD, MA 01201-5423  Race:  W Sex: F  Narrative  Ma  PC 2CS815 repossessed from [i Adams Rd.  
<re.Match object; span=(7, 11), match='0909'> None
19-95  0909  Initiated - BUILDING CHECK  BUILDING CH

In [219]:
parsed_pages = pd.DataFrame(parsed_pages, columns = ['current_date', 'page_num', 'call_number', 'call_time', 'call_reason_action', 'call_taker',"arvd_time", "clrd_time"])                

# cleanup call_takers
parsed_pages['call_taker'] = parsed_pages['call_taker'].str.replace(":", "").str.strip()

parsed_pages.to_csv('../../data/parsed_logs_2019.csv', mode='w', index=False, header=True)

parsed_pages


,current_date,page_num,call_number,call_time,call_reason_action,call_taker,arvd_time,clrd_time
0,01/01/2019,1,19-1,0341,Initiated - PARKING CHECK SERVICES RENDERED,PATROL CRAIG A EICHHAMMER,03:42:00,03:42:17
1,01/01/2019,1,19-4,0834,Initiated - BUILDING CHECK BUILDING CHECKED/S...,"PATROL DAVID JENNINGS, D",08:34:38,08:35:33
2,01/01/2019,1,19-5,0842,Initiated - BUILDING CHECK BUILDING CHECKED/S...,"PATROL DAVID JENNINGS, D",08:43:07,08:43:21
3,01/01/2019,1,19-6,0846,Initiated - BUILDING CHECK BUILDING CHECKED/S...,"PATROL DAVID JENNINGS, D",08:46:51,08:47:07
4,01/01/2019,1,19-7,0847,Initiated - BUILDING CHECK BUILDING CHECKED/S...,"PATROL DAVID JENNINGS, D",08:47:37,08:48:05
...,...,...,...,...,...,...,...,...
90,01/03/2019,18,19-118,1729,Initiated - BUILDING CHECK BUILDING CHECKED/S...,SERGEANT DAVID R LEMIEUX,17:29:15,17:30:26
91,01/03/2019,19,19-121,1839,Initiated - BUILDING CHECK BUILDING CHECKED/S...,SERGEANT DAVID R LEMIEUX,18:40:41,~18:43:03
92,01/03/2019,19,19-125,1945,Initiated - SUSPICIOUS. MOTOR VEHICLE LOG ENT...,PATROL KEVIN P GARNER,19:48:30,19:49:03
93,01/03/2019,19,19-126,2044,Initiated - SUSPICIOUS MOTOR VEHICLE LOG ENTR...,PATROL KEVIN P GARNER,20:48:57,20:49:39


In [220]:
parsed_pages.iloc[-30:]

,current_date,page_num,call_number,call_time,call_reason_action,call_taker,arvd_time,clrd_time
65,01/03/2019,14,19-92,os0o,5 Initiated - BUILDING CHECK BUILDING CHECKE...,PATROL TANIA HERNANDEZ,08:05:19,08:05:39
66,01/03/2019,14,19-94,0807,Other ~ ASSIST OTHER AGENCY - REPO SERVICES R...,None,None,None
67,01/03/2019,14,19-93,0808,Initiated - TRAFFIC CONTROL SERVICES RENDERED,PATROL TANIA HERNANDEZ,08:09:01,~08:38:29
68,01/03/2019,14,19-95,0909,,PATROL TANIA HERNANDEZ,09:09:18,03:09:50
69,01/03/2019,15,19-96,0912,Initiated - BUILDING CHECK BUILDING CHECKED/S...,PATROL TANIA HERNANDEZ,None,~-09:13:44
70,01/03/2019,15,19-97,0914,Initiated - BUILDING CHECK BUILDING CHECKED/S...,PATROL TANIA HERNANDEZ,09:14:23,09:14:58
71,01/03/2019,15,19-98,0918,Initiated - PARKING CHECK SERVICES RENDERED,PATROL TANIA HERNANDEZ,09:19:01,09:25:41
72,01/03/2019,15,19-99,0922,Initiated - BUILDING CHECK BUILDING CHECKED/S...,PATROLMAN MICHAEL J ZIEMBA Jr,09:22:56,09:
73,01/03/2019,15,19-100,0927,Initiated ~ PARKING CHECK SERVICES RENDERED,PATROL TANIA HERNANDEZ,09:27:55,09:33:27
74,01/03/2019,16,19-101,0928,. Initiated - BUILDING CHECK BUILDING CHECKE...,PATROLMAN MICHAEL J ZIEMBA Jr,09:28:20,09:28:32
